<a href="https://colab.research.google.com/github/DrRagasachi/DrRagasachi/blob/main/convertidor_pdf-url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests tabula-py pandas jpype1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 4.6 MB/s eta 0:00:00


In [6]:
# Importa las bibliotecas
import requests
import pandas as pd
import tabula
import os

def pdf_a_dataframe(url):
    """
    Descarga un PDF desde una URL, extrae las tablas y las combina en un único DataFrame.

    Parámetros:
        url (str): La URL del archivo PDF.

    Retorna:
        pd.DataFrame: Un DataFrame con todas las tablas combinadas. Si no se encuentran tablas o hay un error, retorna un DataFrame vacío.
    """
    try:
        # Descarga el PDF desde la URL
        respuesta = requests.get(url)
        respuesta.raise_for_status()  # Lanza una excepción si hay un error HTTP

        # Guarda el PDF en un archivo temporal
        with open("temp.pdf", "wb") as archivo:
            archivo.write(respuesta.content)

        # Extrae las tablas del archivo PDF
        tablas = tabula.read_pdf("temp.pdf", pages='all')

        # Si no se encuentran tablas, retorna un DataFrame vacío
        if not tablas:
            print("No se encontraron tablas en el PDF.")
            return pd.DataFrame()

        # Inicializa un DataFrame vacío para almacenar las tablas combinadas
        dataframe_combinado = pd.DataFrame()

        # Combina todas las tablas en un único DataFrame
        for tabla in tablas:
            df = tabla.copy()
            dataframe_combinado = pd.concat([dataframe_combinado, df], ignore_index=True)

        # Elimina el archivo temporal
        os.remove("temp.pdf")

        return dataframe_combinado

    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el PDF: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al procesar el PDF: {e}")
        return pd.DataFrame()

# URL del archivo PDF que deseas convertir
url_pdf = 'https://drive.google.com/file/d/1-IBIorkwi2GD0i_D4IQrOIJHeClK8FkF/view?usp=drive_link'

# Llama a la función para convertir el PDF en un DataFrame
dataframe_combinado = pdf_a_dataframe(url_pdf)

# Verifica el resultado
if not dataframe_combinado.empty:
    print("Datos extraídos correctamente:")
    print(dataframe_combinado.head())  # Muestra las primeras filas del DataFrame

    # Guarda el DataFrame en un archivo Excel
    nombre_archivo = "resultados.xlsx"
    dataframe_combinado.to_excel(nombre_archivo, index=False)
    print(f"El archivo '{nombre_archivo}' se ha guardado correctamente.")

    # Descarga el archivo Excel desde Colab
    from google.colab import files
    files.download(nombre_archivo)
else:
    print("No se pudo extraer ningún dato del PDF.")

Error al descargar el PDF: 401 Client Error: Unauthorized for url: https://drive.google.com/file/d/1-IBIorkwi2GD0i_D4IQrOIJHeClK8FkF/view?usp=drive_link
No se pudo extraer ningún dato del PDF.
